In [42]:
import pandas_datareader as wb
import pandas as pd
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [55]:
# 코스피지수 크롤링
#약 3년치 데이터를 크롤링해 옴
start = datetime.datetime(2019, 8, 1)
# 현재 날짜까지
end = date.today()

# ^KS11 : 코스피
df_null = wb.DataReader("^KS11","yahoo",start,end)

# 결측치 제거
df = df_null.dropna()

# Close와 Adj Close는 중복되는 columns인것을 확인 함
# Adj Close열을 제거
df.drop(["Adj Close"],axis=1, inplace=True)

# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(len(df)-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# columns명을 알기 쉽게 한글로 변경
df.columns = ["최고가", "최저가" , "시작가", "종가", "거래량" , "등락"]

# 파일 저장
df.to_csv('kospi_주가데이터.csv',encoding='utf-8-sig')


/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_22593/3761846299.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 1
/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_22593/3761846299.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 0


,최고가,최저가,시작가,종가,거래량,등락
Date,,,,,,
2022-11-14,2499.429932,2470.600098,2485.179932,2474.649902,730900,1
2022-11-15,2485.199951,2463.649902,2482.760010,2480.330078,617100,0
2022-11-16,2487.000000,2446.790039,2487.000000,2477.449951,657600,0
2022-11-17,2467.389893,2442.899902,2466.500000,2442.899902,914519,0


In [56]:
# 등락 값이 "0"인 데이터와 "1"인 데이터를 나눠서 리스트화
# 여기서 리스트화 하는 이유는 다음날 주가가 하락한 경우의 뉴스 타이틀과 상승한 뉴스타이틀을 크롤링해 분류하기 위함
price_data = pd.read_csv('kospi_주가데이터.csv')

#주가가 하락한 경우의 Date 데이터 리스트 date_0
df_0 = price_data[price_data['등락']==0]['Date']
date_0 = []
for i in range(0,len(df_0)):
    date_0.append(str(df_0.tolist()[i])[:10].replace('-',''))

#주가가 상승한 경우의 Date 데이터 리스트 date_1
df_1 = price_data[price_data['등락']==1]['Date']
date_1 = []
for i in range(0,len(df_1)):
    date_1.append(str(df_1.tolist()[i])[:10].replace('-',''))

['20221114']

In [45]:
#뉴스타이틀 크롤링을 위한 모듈 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse

In [58]:
# 뉴스타이틀 크롤링한 뒤 제목 리스틑 리턴
def news_title_crawling(dates, base_url, news_block, news_title_block) -> list:
    result_list = []
    error_cnt = 0
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(date, page)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select(news_block)
                for title in title_list:
                    try:
                        news_title = title.select_one(news_title_block).text.strip()
                        find_1 = news_title.find("[")
                        find_2 = news_title.find("]")
                        if find_1 != -1:
                            slice_news_title = news_title[find_1:find_2+1]
                            news_title.strip(slice_news_title)
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        elif find_1 == -1:
                            result_list.append([news_title])
                    except:
                        error_cnt += 1
    return result_list

In [59]:
# 뉴스 타이틀 크롤링한 리스트 병합 후 csv파일로 변환
def news_title_to_csv(site_name:str, date_0, date_1, base_url, news_block, news_title_block):
    result_list = news_title_crawling(date_0, base_url, news_block, news_title_block)
    title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
    title_df_0['주가변동'] = 0

    result_list = news_title_crawling(date_1, base_url, news_block, news_title_block)
    title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
    title_df_1['주가변동'] = 1

    # 크롤링 데이터 합쳐 csv파일로 만들기
    title_df = pd.concat([title_df_0, title_df_1])
    # 중복 데이터 삭제 
    title_df.drop_duplicates(['뉴스제목'])
    # 데이터프레임 저장 
    title_df.to_csv(site_name + '_뉴스타이틀.csv', index=False, encoding='utf-8-sig')

In [62]:
# news_title_to_csv(site_name:str, date_0, date_1, base_url, news_block, news_title_block)
news_title_to_csv("팍스넷", date_0, date_1,
                base_url='http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={1}&genDate={0}&objId=N4667',
                news_block='ul.thumb-list li', 
                news_title_block='dl.text > dt')
news_title_to_csv("네이버", date_0, date_1,
                base_url='https://finance.naver.com/news/mainnews.naver?date={0}&page={1}',
                news_block='#contentarea_left > div.mainNewsList > ul > li > dl',
                news_title_block='.articleSubject')
news_title_to_csv("다음", date_0, date_1,
                base_url='https://news.daum.net/breakingnews/economic/stock?page={1}&regDate={0}',
                news_block='ul.list_news2.list_allnews > li',
                news_title_block='.link_txt')
news_title_to_csv("네이트", date_0, date_1,
                base_url='https://news.nate.com/subsection?cate=eco06&mid=n0307&type=c&date={0}&page={1}',
                news_block='div.postSubject > ul > li',
                news_title_block='a')

In [64]:
#뉴스데이터 하나로 합치기 
paxnet_data = pd.read_csv('팍스넷_뉴스타이틀.csv')
naver_data = pd.read_csv('네이버_뉴스타이틀.csv')
daum_data = pd.read_csv('다음_뉴스타이틀.csv')
nate_data = pd.read_csv('네이트_뉴스타이틀.csv')
all_title = pd.concat([naver_data, paxnet_data, daum_data, nate_data])
all_title.to_csv('news_titles.csv', index=False, encoding='utf-8-sig')

In [50]:
# # 팍스넷 크롤링 함수 paxnet_news_title
# result_list = []
# error_cnt = 0

# def paxnet_news_title(dates):
#     base_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
#     headers = {
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
#     }
    
#     for date in dates:
#         for page in range(1, 3):
#             url = base_url.format(page, date)
#             res = requests.get(url, headers=headers)
#             if res.status_code == 200:
#                 soup = BeautifulSoup(res.text)
#                 title_list = soup.select('ul.thumb-list li')
#                 for title in title_list:
#                     try:
#                         news_title = title.select_one('dl.text > dt').text.strip()
#                         #"[문자]" 가 붙은 제거
#                         #find 함수는 문자열안에 지정한 문자가 있을경우 그 위치(index)를 int로 반환해준다.
#                         #지정한 문자를 찾지 못할 경우 -1 을 반환 
#                         find_1=news_title.find("[")
#                         find_2=news_title.find("]")

#                         # find함수의 결과가 -1가 아닌 경우(문자열안에 "[문자]"가 있다.)
#                         # 그 위치를 찾아 제거하고 result_list에 추가
#                         if find_1 != -1:
#                             slice_news_title=news_title[find_1:find_2+1]
#                             result_list.append([news_title.strip(slice_news_title).strip()])
#                         #find함수의 결과가 -1 일경우(문자열안에 "[문자]"가 없다.)
#                         #바로 result_list에 추가
#                         elif find_1 == -1:
#                             result_list.append([news_title])
#                     except:
#                         error_cnt += 1
# paxnet_news_title(date_0)
# title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_0['주가변동'] = 0
# result_list = []

# paxnet_news_title(date_1)
# title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_1['주가변동'] = 1
# result_list = []



# #팍스넷 크롤링 데이터 합쳐 csv파일로 만들기
# paxnet_title_df = pd.concat([title_df_0, title_df_1])
# #중복 데이터 삭제 
# paxnet_title_df.drop_duplicates(['뉴스제목'])
# #데이터프레임 저장 
# paxnet_title_df.to_csv('팍스넷_뉴스타이틀.csv', index=False, encoding='utf-8-sig')

In [51]:
# # 네이버 크롤링 함수 naver_news_title
# result_list = []
# error_cnt = 0
# def naver_news_title(dates):
#     base_url = 'https://finance.naver.com/news/mainnews.naver?date={}&page={}'
#     headers = {
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
#     }
#     for date in dates:
#         for page in range(1, 3):
#             url = base_url.format(date,page)
#             res = requests.get(url, headers=headers)
#             if res.status_code == 200:
#                 soup = BeautifulSoup(res.text)
#                 title_list = soup.select('#contentarea_left > div.mainNewsList > ul > li > dl')
#                 for title in title_list:
#                     try:
#                         news_title = title.select_one('.articleSubject').text.strip()
#                         find_1=news_title.find("[")
#                         find_2=news_title.find("]")
#                         if find_1 != -1:
#                             slice_news_title=news_title[find_1:find_2+1]
#                             news_title.strip(slice_news_title)
#                             result_list.append([news_title.strip(slice_news_title).strip()])
#                         elif find_1 == -1:
#                             result_list.append([news_title])
#                     except:
#                         error_cnt += 1

# naver_news_title(date_0)
# title_df_2 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_2['주가변동'] = 0
# result_list = []

# naver_news_title(date_1)
# title_df_3 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_3['주가변동'] = 1
# result_list = []

# #네이버 크롤링 데이터 합쳐 csv파일로 만들기
# naver_title_df = pd.concat([title_df_2, title_df_3])
# #중복 데이터 삭제 
# naver_title_df.drop_duplicates(['뉴스제목'])
# naver_title_df.to_csv('네이버_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
# naver_title_df


In [52]:
# # 다음 크롤링 함수 daum_news_title
# result_list = []
# error_cnt = 0
# def daum_news_title(dates):
#     base_url = "https://news.daum.net/breakingnews/economic/stock?page={1}&regDate={0}"
#     headers = {
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
#     }
#     for date in dates:
#         for page in range(1, 3):
#             url = base_url.format(date, page)
#             res = requests.get(url, headers=headers)
#             if res.status_code == 200:
#                 soup = BeautifulSoup(res.text)
#                 title_list = soup.select('ul.list_news2.list_allnews > li')
#                 for title in title_list:
#                     try:
#                         news_title = title.select_one('.link_txt').text.strip()
#                         find_1=news_title.find("[")
#                         find_2=news_title.find("]")
#                         if find_1 != -1:
#                             slice_news_title=news_title[find_1:find_2+1]
#                             news_title.strip(slice_news_title)
#                             result_list.append([news_title.strip(slice_news_title).strip()])
#                         elif find_1 == -1:
#                             result_list.append([news_title])
#                     except:
#                         error_cnt += 1

# daum_news_title(date_0)
# title_df_4 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_4['주가변동'] = 0
# result_list = []

# daum_news_title(date_1)
# title_df_5 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_5['주가변동'] = 1
# result_list = []

# #다음 크롤링 데이터 합쳐 csv파일로 만들기
# daum_title_df = pd.concat([title_df_4, title_df_5])
# #중복 데이터 삭제 
# daum_title_df.drop_duplicates(['뉴스제목'])
# daum_title_df.to_csv('다음_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
# daum_title_df


In [53]:
# # 네이트 크롤링 함수 nate_news_title
# result_list = []
# error_cnt = 0
# def nate_news_title(dates):
#     base_url = "https://news.nate.com/subsection?cate=eco04&mid=n0304&type=t&date={0}"
#     headers = {
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
#     }
#     for date in dates:
#         url = base_url.format(date)
#         res = requests.get(url, headers=headers)
#         if res.status_code == 200:
#             soup = BeautifulSoup(res.text)
#             title_list = soup.select('div.postSubject > ul > li')
#             for title in title_list:
#                 try:
#                     news_title = title.select_one('a').text.strip()
#                     find_1=news_title.find("[")
#                     find_2=news_title.find("]")
#                     if find_1 != -1:
#                         slice_news_title=news_title[find_1:find_2+1]
#                         news_title.strip(slice_news_title)
#                         result_list.append([news_title.strip(slice_news_title).strip()])
#                     elif find_1 == -1:
#                         result_list.append([news_title])
#                 except:
#                     error_cnt += 1

# nate_news_title(date_0)
# title_df_6 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_6['주가변동'] = 0
# result_list = []

# nate_news_title(date_1)
# title_df_7 = pd.DataFrame(result_list, columns=['뉴스제목'])
# title_df_7['주가변동'] = 1
# result_list = []

# #네이트 크롤링 데이터 합쳐 csv파일로 만들기
# nate_title_df = pd.concat([title_df_6, title_df_7])
# #중복 데이터 삭제 
# nate_title_df.drop_duplicates(['뉴스제목'])
# nate_title_df.to_csv('네이트_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
# nate_title_df